<a href="https://colab.research.google.com/github/novrian6/product_recommendation_engine/blob/main/product_recommendation_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from surprise import Dataset, Reader
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the CSV files
category_tree = pd.read_csv('category_tree.csv')
events = pd.read_csv('events.csv')
item_properties_part1 = pd.read_csv('item_properties_part1.csv')
item_properties_part2 = pd.read_csv('item_properties_part2.csv')

# Merging the data
item_properties = pd.concat([item_properties_part1, item_properties_part2])
merged_data = events.merge(item_properties, on='itemid', how='left')

# Selecting relevant columns
merged_data = merged_data[['timestamp_x', 'visitorid', 'event', 'itemid', 'transactionid', 'timestamp_y', 'property', 'value']]

# Create a 'rating' column, for simplicity we'll just use the event 'view'
merged_data['rating'] = merged_data['event'].apply(lambda x: 1 if x == 'view' else 0)

# Define the reader
reader = Reader(rating_scale=(0, 1))

# Load the data into the Surprise format
data = Dataset.load_from_df(merged_data[['visitorid', 'itemid', 'rating']], reader)

# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Use the SVD algorithm for recommendations
model = SVD()
model.fit(trainset)

# Make predictions on the test data
predictions = model.test(testset)

# Calculate and print the RMSE (Root Mean Squared Error)
rmse = accuracy.rmse(predictions)
print("RMSE:", rmse)

# Recommend items for a specific user (for example, visitorid 12345)
def recommend_items(user_id, model, num_items=10):
    items = []
    for item_id in range(1, merged_data['itemid'].nunique() + 1):
        if not model.predict(user_id, item_id).details['was_impossible']:
            items.append((item_id, model.predict(user_id, item_id).est))
    items.sort(key=lambda x: x[1], reverse=True)
    recommended_items = items[:num_items]
    return recommended_items

# Example: Recommend items for visitorid 12345
visitor_id = 12345
recommendations = recommend_items(visitor_id, model)
print(f"\nTop {len(recommendations)} Recommendations for VisitorID {visitor_id}:")
for item_id, score in recommendations:
    print(f"ItemID: {item_id}, Score: {score}")


In [2]:
!pip install surprise
!pip install panda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162992 sha256=3078f5107a65bb4386d3669b10f17c2c56403e36df926ef8f7cb0fb3d3874079
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [7]:
merged_data

,timestamp_x,visitorid,event,itemid,transactionid,timestamp_y,property,value
0,1433221332117,257597.0,view,355908.0,NaN,1.431832e+12,159,519769
1,1433221332117,257597.0,view,355908.0,NaN,1.431832e+12,available,1
2,1433221332117,257597.0,view,355908.0,NaN,1.440904e+12,available,1
3,1433221332117,257597.0,view,355908.0,NaN,1.436065e+12,available,1
4,1433221332117,257597.0,view,355908.0,NaN,1.437880e+12,available,1
...,...,...,...,...,...,...,...,...
7217820,1441416367857,427135.0,view,302329.0,NaN,1.431227e+12,6,147181
7217821,1441416367857,427135.0,view,302329.0,NaN,1.433646e+12,761,n79.200 190776
7217822,1441416367857,427135.0,view,302329.0,NaN,1.432436e+12,573,1128577 838923
7217823,1441416367857,427135.0,view,302329.0,NaN,1.433041e+12,283,740324 727528 1050009 1180709 237903 251836 12...
